In [1]:
import pandas as pd
import numpy as np

In [16]:
cds_data = pd.read_csv('cds3.csv', index_col=0, parse_dates=True)
#   Convert the data from bps to percentage
cds_data = cds_data / 10000
#   Data cleaning
cds_data = cds_data.ffill().bfill()
cds_data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5194 entries, 2003-05-15 to 2023-04-11
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Bank of America Corp                  5194 non-null   float64
 1   Boeing Co/The                         5194 non-null   float64
 2   Simon Property Group LP               5194 non-null   float64
 3   Citigroup Inc                         5194 non-null   float64
 4   JPMorgan Chase & Co                   5194 non-null   float64
 5   International Business Machines Corp  5194 non-null   float64
 6   Motorola Solutions Inc                5194 non-null   float64
 7   Caterpillar Inc                       5194 non-null   float64
 8   Nestle SA (EUR)                       5194 non-null   float64
 9   McDonald's Corp                       5194 non-null   float64
dtypes: float64(10)
memory usage: 446.4 KB


In [ ]:
def calculate_default_intensity(cds_data, data_range = 252, recovery_rate=0.4):
    cds_data = cds_data.iloc[-data_range:]
    intensity = pd.DataFrame(index=cds_data.index, columns=cds_data.columns)

    for cds in intensity.columns:
        intensity[cds] = cds_data[cds] / (1 - recovery_rate)

    cds_mu = intensity.mean()
    #
    cds_sigma = intensity.std()

    cds_corr = intensity.corr()
        
    return intensity, cds_mu, cds_sigma, cds_corr

def calculate_default_prob(cds_data, intensity, data_range = 252, time_horizon=1/252):
    cds_data = cds_data.iloc[-data_range:]
    prob_default = pd.DataFrame(index=cds_data.index, columns=cds_data.columns)
    
    for cds in prob_default.columns:
        prob_default[cds] = 1 - np.exp(-intensity[cds] * time_horizon)
    
    prob_default = prob_default.ffill().bfill()
    
    return prob_default

intensity, cds_mu, cds_sigma, cds_corr = calculate_default_intensity(cds_data, data_range = 252, recovery_rate=0.4)
prob_default = calculate_default_prob(cds_data, intensity, time_horizon=1)

initial_intensity = intensity.iloc[-1]



In [65]:
np.random.seed(42)
selected_components = np.random.choice(cds_data.columns[:-1], 3, replace=False)
counterparty = np.random.choice(cds_data.columns[-1:], 1)[0]

selected = list(selected_components) + [counterparty]
mu = cds_mu[selected]
sigma = cds_sigma[selected]
corr = cds_corr.loc[selected, selected]
X = initial_intensity[selected]